In [297]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from matplotlib import pyplot as plt
%matplotlib inline
import pandas_profiling as pp
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
import glob
import json
import folium
import squarify
from datetime import date
import datetime
import re
import nltk
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
import regex
import random
from sklearn.utils import shuffle
from datetime import date
import datetime
pd.options.display.max_columns = 100

In [298]:
df =  pd.read_csv(r"N:\CLIX\Perfios\Perfios_rawdata_complete.csv",encoding='latin-1')
df3 = df[df['TransDate'] <= df['enddate']] 
df = df3[df3['TransDate'] >= df3['startdate']]
df['startdate']=pd.to_datetime(df['startdate'],unit ='s')
df['enddate']=pd.to_datetime(df['enddate'],unit = 's')
df['TransDate']=pd.to_datetime(df['TransDate'],unit = 's')
df['Year'] = pd.DatetimeIndex(df['TransDate']).year
df['Month'] = pd.DatetimeIndex(df['TransDate']).month
df['Day'] = pd.DatetimeIndex(df['TransDate']).day
df["Credited"] = df.Amount.where(df.Amount > 0, 0)
df["Debited"] = -1 * df.Amount.where(df.Amount < 0, 0) 
df['Debited'] = df['Debited'].abs()
df['Balance'] = df['Balance'].abs()

In [299]:
df = df.sort_values(['userid','TransDate','BankNarrationtId'],ascending= True)
df_1 = df.groupby(['userid','TransDate','BankAccountId'])['Debited','Credited'].sum()
df_1 = pd.DataFrame(df_1)
df_1 = df_1.reset_index()

In [300]:
df_4 = df.groupby(['userid','TransDate','BankAccountId'])['BankNarrationtId'].min()
df_4 = pd.DataFrame(df_4)
df_4 = df_4.reset_index()

In [301]:
df_5 = df.groupby(['userid','TransDate','BankAccountId'])['BankNarrationtId'].max()
df_5 = pd.DataFrame(df_5)
df_5 = df_5.reset_index()

In [302]:
#df_5 = pd.merge(df_1,df_5,on = ['userid','TransDate'],how = 'left')

In [303]:
df_5 = pd.merge(df_5,df[['userid','TransDate','BankNarrationtId','Balance','BankAccountId']],how = 'left',on = ['userid','TransDate','BankNarrationtId','BankAccountId'])

In [304]:
df_4 = pd.merge(df_4,df[['userid','TransDate','BankNarrationtId','Balance','BankAccountId']],how = 'left',on = ['userid','TransDate','BankNarrationtId','BankAccountId'])

In [305]:
df_6 = pd.merge(df_4,df_5,how = 'left',on = ['userid','TransDate','BankAccountId'])
df_6.rename(columns = {"BankNarrationtId_x": "BankNarrationtId_start","BankNarrationtId_y": "BankNarrationtId_end","Balance_y":"Day_end_Balance","Balance_x":"Day_start_Balance"}, inplace = True)

In [306]:
df_6

,userid,TransDate,BankAccountId,BankNarrationtId_start,Day_start_Balance,BankNarrationtId_end,Day_end_Balance
0,4685,2016-10-01,170,14871,24805.97,14876,16305.97
1,4685,2016-10-01,171,14722,12666.94,14726,2166.94
2,4685,2016-10-02,170,14877,16300.22,14877,16300.22
3,4685,2016-10-02,171,14727,1836.94,14728,813.94
4,4685,2016-10-03,170,14878,7223.22,14881,6595.22
5,4685,2016-10-03,171,14729,0.94,14731,0.94
6,4685,2016-10-04,170,14882,6589.47,14883,5089.47
7,4685,2016-10-05,170,14884,89.47,14884,89.47
8,4685,2016-10-06,170,14885,83.72,14887,283.72
9,4685,2016-10-07,170,14888,49583.72,14893,21667.02


In [307]:
df_dummy = df.iloc[:,[2,3,4,10]]
df_dummy = df_dummy.drop_duplicates()
df_dummy

,userid,startdate,enddate,BankAccountId
3495,4685,2016-10-01,2016-12-31,171
3472,4685,2016-10-01,2016-12-31,170
2713,9093,2016-10-01,2016-12-31,149
4340,10982,2017-04-01,2017-06-30,1697
223,17123,2017-07-01,2017-09-30,1992
4647,22062,2017-03-01,2017-05-31,1241
3754,23167,2017-02-01,2017-04-30,1056
5598,23693,2016-12-01,2017-02-28,653
208,28126,2016-10-01,2016-12-31,153
174,28382,2017-02-01,2017-04-30,1117


In [308]:
func = lambda x: pd.date_range(x.startdate, x.enddate).to_series()

In [309]:
df_dummy = df_dummy.set_index(['userid', 'BankAccountId'])

In [310]:
df_dummy

startdate    enddate
userid BankAccountId                      
4685   171           2016-10-01 2016-12-31
       170           2016-10-01 2016-12-31
9093   149           2016-10-01 2016-12-31
10982  1697          2017-04-01 2017-06-30
17123  1992          2017-07-01 2017-09-30
22062  1241          2017-03-01 2017-05-31
23167  1056          2017-02-01 2017-04-30
23693  653           2016-12-01 2017-02-28
28126  153           2016-10-01 2016-12-31
28382  1117          2017-02-01 2017-04-30
34240  471           2016-11-01 2017-01-31
34337  834           2016-12-01 2017-02-28
38874  1156          2017-02-01 2017-04-30
41916  1392          2017-03-01 2017-05-31
43733  1254          2017-03-01 2017-05-31
58688  291           2016-11-01 2017-01-31
67675  1223          2017-02-01 2017-04-30
69086  1687          2017-04-01 2017-06-30
72310  121           2016-10-01 2016-12-31
73289  1729          2017-04-01 2017-06-30
74104  771           2016-12-01 2017-02-28
75338  502           2016-11-01 2017-01-31
       503           2016-11-01 2017-01-31
76240  1599          2017-03-01 2017-05-31
       1598          2017-03-01 2017-05-31
81152  1978          2017-04-01 2017-06-30
83100  1607          2017-04-01 2017-06-30
86148  147           2016-10-01 2016-12-31
90325  1135          2017-02-01 2017-04-30
90347  145           2016-11-01 2017-01-31
...                         ...        ...
678819 2020          2017-04-01 2017-06-30
678927 2023          2017-04-01 2017-06-30
679171 2025          2017-04-01 2017-06-30
       2026          2017-04-01 2017-06-30
679894 2029          2017-05-01 2017-07-31
681090 2032          2017-04-01 2017-06-30
681822 2034          2017-04-01 2017-07-31
682463 2067          2017-05-01 2017-07-31
682490 2037          2017-04-01 2017-07-31
682793 2073          2017-05-01 2017-07-31
683075 2044          2017-04-01 2017-07-31
683795 2039          2017-04-01 2017-07-31
684839 2042          2017-04-01 2017-07-31
686670 2049          2017-05-01 2017-07-31
       2050          2017-05-01 2017-07-31
688111 2051          2017-05-01 2017-07-31
689231 2053          2017-05-01 2017-07-31
689705 2054          2017-05-01 2017-07-31
689928 2055          2017-05-01 2017-07-31
690315 2058          2017-05-01 2017-07-31
692232 2070          2017-05-01 2017-07-31
692460 2066          2017-05-01 2017-07-31
693258 2064          2017-05-01 2017-07-31
693384 2074          2017-05-01 2017-07-31
       2075          2017-05-01 2017-07-31
695642 2076          2017-05-01 2017-07-31
695814 2078          2017-05-01 2017-07-31
696301 2089          2017-05-01 2017-07-31
696624 2082          2017-05-01 2017-07-31
697295 2081          2017-05-01 2017-07-31

[1493 rows x 2 columns]

In [311]:
y = df_dummy.apply(func,axis=1)
y

2016-08-01 2016-08-02 2016-08-03 2016-08-04 2016-08-05  \
userid BankAccountId                                                          
4685   171                  NaT        NaT        NaT        NaT        NaT   
       170                  NaT        NaT        NaT        NaT        NaT   
9093   149                  NaT        NaT        NaT        NaT        NaT   
10982  1697                 NaT        NaT        NaT        NaT        NaT   
17123  1992                 NaT        NaT        NaT        NaT        NaT   
22062  1241                 NaT        NaT        NaT        NaT        NaT   
23167  1056                 NaT        NaT        NaT        NaT        NaT   
23693  653                  NaT        NaT        NaT        NaT        NaT   
28126  153                  NaT        NaT        NaT        NaT        NaT   
28382  1117                 NaT        NaT        NaT        NaT        NaT   
34240  471                  NaT        NaT        NaT        NaT        NaT   
34337  834                  NaT        NaT        NaT        NaT        NaT   
38874  1156                 NaT        NaT        NaT        NaT        NaT   
41916  1392                 NaT        NaT        NaT        NaT        NaT   
43733  1254                 NaT        NaT        NaT        NaT        NaT   
58688  291                  NaT        NaT        NaT        NaT        NaT   
67675  1223                 NaT        NaT        NaT        NaT        NaT   
69086  1687                 NaT        NaT        NaT        NaT        NaT   
72310  121                  NaT        NaT        NaT        NaT        NaT   
73289  1729                 NaT        NaT        NaT        NaT        NaT   
74104  771                  NaT        NaT        NaT        NaT        NaT   
75338  502                  NaT        NaT        NaT        NaT        NaT   
       503                  NaT        NaT        NaT        NaT        NaT   
76240  1599                 NaT        NaT        NaT        NaT        NaT   
       1598                 NaT        NaT        NaT        NaT        NaT   
81152  1978                 NaT        NaT        NaT        NaT        NaT   
83100  1607                 NaT        NaT        NaT        NaT        NaT   
86148  147                  NaT        NaT        NaT        NaT        NaT   
90325  1135                 NaT        NaT        NaT        NaT        NaT   
90347  145                  NaT        NaT        NaT        NaT        NaT   
...                         ...        ...        ...        ...        ...   
678819 2020                 NaT        NaT        NaT        NaT        NaT   
678927 2023                 NaT        NaT        NaT        NaT        NaT   
679171 2025                 NaT        NaT        NaT        NaT        NaT   
       2026                 NaT        NaT        NaT        NaT        NaT   
679894 2029                 NaT        NaT        NaT        NaT        NaT   
681090 2032                 NaT        NaT        NaT        NaT        NaT   
681822 2034                 NaT        NaT        NaT        NaT        NaT   
682463 2067                 NaT        NaT        NaT        NaT        NaT   
682490 2037                 NaT        NaT        NaT        NaT        NaT   
682793 2073                 NaT        NaT        NaT        NaT        NaT   
683075 2044                 NaT        NaT        NaT        NaT        NaT   
683795 2039                 NaT        NaT        NaT        NaT        NaT   
684839 2042                 NaT        NaT        NaT        NaT        NaT   
686670 2049                 NaT        NaT        NaT        NaT        NaT   
       2050                 NaT        NaT        NaT        NaT        NaT   
688111 2051                 NaT        NaT        NaT        NaT        NaT   
689231 2053                 NaT        NaT        NaT        NaT        NaT   
689705 2054                 NaT        NaT        NaT        NaT       

In [312]:
z = y.stack()

In [313]:
z = pd.DataFrame(z)
z = z.reset_index()
z

,userid,BankAccountId,level_2,0
0,4685,171,2016-10-01,2016-10-01
1,4685,171,2016-10-02,2016-10-02
2,4685,171,2016-10-03,2016-10-03
3,4685,171,2016-10-04,2016-10-04
4,4685,171,2016-10-05,2016-10-05
5,4685,171,2016-10-06,2016-10-06
6,4685,171,2016-10-07,2016-10-07
7,4685,171,2016-10-08,2016-10-08
8,4685,171,2016-10-09,2016-10-09
9,4685,171,2016-10-10,2016-10-10


In [314]:
z = z.sort_values(['userid','BankAccountId'],ascending = True)

In [315]:
#z = z.drop(0,axis=1)

In [316]:
z =z.rename(columns = {"level_2": "TransDate"})

In [317]:
z[z['userid']==697295]

,userid,BankAccountId,TransDate,0
136021,697295,2081,2017-05-01,2017-05-01
136022,697295,2081,2017-05-02,2017-05-02
136023,697295,2081,2017-05-03,2017-05-03
136024,697295,2081,2017-05-04,2017-05-04
136025,697295,2081,2017-05-05,2017-05-05
136026,697295,2081,2017-05-06,2017-05-06
136027,697295,2081,2017-05-07,2017-05-07
136028,697295,2081,2017-05-08,2017-05-08
136029,697295,2081,2017-05-09,2017-05-09
136030,697295,2081,2017-05-10,2017-05-10


In [320]:
final = pd.merge(df_6,z,on = ['userid','BankAccountId','TransDate'],how = 'outer')

In [230]:
final['Day_start_Balance'] = final['Day_start_Balance'].fillna(0)
final['Day_end_Balance'] = final['Day_end_Balance'].fillna(0)

In [105]:
#final.to_csv(r'N:\CLIX\Perfios\Monthly_avg_bal\Final_avg_mon_balance.csv',index = False)

In [116]:
#final_1 = pd.merge(final,df[['userid','TransDate','BankAccountId','startdate','enddate']],how = 'left',on = ['userid','TransDate','BankAccountId'])

In [324]:
final[(final['userid']== 697295)]# & (final['TransDate']== '2017-05-20')]

,userid,TransDate,BankAccountId,BankNarrationtId_start,Day_start_Balance,BankNarrationtId_end,Day_end_Balance,0
57229,697295,2017-05-20,2081,187878.0,12420.0,187879.0,5420.0,2017-05-20


In [322]:
df_6[df_6['userid']== 697295]

,userid,TransDate,BankAccountId,BankNarrationtId_start,Day_start_Balance,BankNarrationtId_end,Day_end_Balance
57226,697295,2017-05-01,2081,187868,11083.21,187870,60.21
57227,697295,2017-05-04,2081,187871,0.00,187871,0.00
57228,697295,2017-05-19,2081,187872,40613.00,187877,22420.00
57229,697295,2017-05-20,2081,187878,12420.00,187879,5420.00
57230,697295,2017-05-22,2081,187880,1420.00,187882,0.00
57231,697295,2017-05-31,2081,187883,1246.00,187888,37.00
57232,697295,2017-06-01,2081,187889,24971.00,187896,8471.00
57233,697295,2017-06-02,2081,187897,471.00,187897,471.00
57234,697295,2017-06-03,2081,187898,68.50,187898,68.50
57235,697295,2017-06-05,2081,187899,0.00,187899,0.00


In [325]:
final = final.sort_values(['userid','BankAccountId', 'TransDate'],ascending = True)

In [328]:
final.to_csv(r'N:\CLIX\Perfios\Monthly_avg_bal\Final_avg_mon_balance.csv',index = 'False')